In [1]:
%matplotlib inline

(chap-waldanteil)=
# Aufgabe 14: Waldanteil berechnen

Es ist so weit! Nun habt ihr alles gelernt um den 4. und finalen Schritt unserer Monte-Carlo Simulation umzusetzen. Ihr habt jetzt die gleiche Ausgangslage, mit der ihr den Themenblock "Datenqualität und Unsicherheiten" gestartet habt. 

Schau dir an dieser Stelle {numref}`monte-carlo-workflow` nochmal an um den ganzen Arbeitsprozess zu überblicken. In diesem Schritt wollen wir also den Anteil der Zeckenstiche pro "Simulationsrunde", sprich pro Loop Iteration, ermitteln. Dazu iterieren wir über die Liste `monte_carlo` und...

1. konvertieren die *DataFrame* in eine *GeoDataFrame* 
2. ermitteln für jeden Simulierten Punkt ob er sich im Wald befindet oder nicht
3. ermitteln der Anteil der Zeckenstiche im Wald
4. speichern diesen Wert in einer neuen Liste (z.B. `anteil_wald`) ab

Startet auch diese Aufgabe mit einem frischen Jupyter Notebook. Mit dem nachstehenden Code bringt kommt ihr auf den Stand, den ich oben beschrieben habe und wir uns gemeinsam erarbeitet haben.

In [2]:

# Benötigten Module laden #####################
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
###############################################

# Notwendige Functions definieren #############
def offset_coordinate(old, distance = 100):
    import random
    new = old + random.normalvariate(0,distance)
    return(new)
###############################################

# Daten Importieren ###########################
zeckenstiche = pd.read_csv("zeckenstiche.csv")
wald = gpd.read_file("wald.gpkg")
###############################################

# Daten Simulieren ############################
monte_carlo = []
for i in range(5):
    zeckenstiche["x_sim"] = zeckenstiche["x"].apply(offset_coordinate)
    zeckenstiche["y_sim"] = zeckenstiche["y"].apply(offset_coordinate)
    monte_carlo.append(zeckenstiche.copy())
###############################################

## Übungen

(ex-14-1)=
### Übung 14.1 

Wie iterieren wir über die Liste `monte_carlo`? Schaue dir nochmals {ref}`forloops-2` an und erstelle ein *For Loop* mit dem Platzhalter `sim` welche über die Liste `monte_carlo` iteriert. In jeder Iteration soll der Datentyp (`type`) von `sim` heraus gegeben werden (`print`). 

In [3]:
# Musterlösung

for sim in monte_carlo:
    print(type(sim))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


```python
# Der Output sollte folgendermassen aussehen:

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
```

(ex-14-2)=
### Übung 14.2

Erweitere den erstellten *For Loop* so, dass in jeder Iteration `sim` in eine *GeoDataFrame* konvertieren wird (gespeichert innerhalb des Loops als `sim_gpd`). Schaue dir {ref}`chap-pythongis` an wenn du nicht mehr weisst, wie die Konvertierung *DataFrame* > *GeoDataFrame* geht.

Zum Schluss des Loops soll der Datentyp von `sim_gpd` herausgegeben werden.

In [7]:
# Musterlösung

for sim in monte_carlo:
    sim_gpd = gpd.GeoDataFrame(sim.copy(),geometry=gpd.points_from_xy(zeckenstiche['x'], zeckenstiche['y']),crs = 2056)

      ID    accuracy        x        y         x_sim         y_sim  \
0   2550  439.128951  2681116  1250648  2.681081e+06  1.250701e+06   
1  10437  301.748542  2681092  1250672  2.681027e+06  1.250676e+06   
2   9174  301.748542  2681128  1250683  2.681099e+06  1.250777e+06   
3   8773  301.748542  2681111  1250683  2.681085e+06  1.250559e+06   
4   2764  301.748529  2681131  1250692  2.681317e+06  1.250629e+06   
5   2513  301.748529  2681171  1250711  2.681120e+06  1.250715e+06   
6   9185  301.748542  2681107  1250712  2.681138e+06  1.250689e+06   
7  28521  301.748542  2681124  1250720  2.681289e+06  1.250726e+06   
8  26745  301.748542  2681117  1250725  2.681098e+06  1.250835e+06   
9  27391  301.748542  2681138  1250725  2.681155e+06  1.250867e+06   

                          geometry  
0  POINT (2681116.000 1250648.000)  
1  POINT (2681092.000 1250672.000)  
2  POINT (2681128.000 1250683.000)  
3  POINT (2681111.000 1250683.000)  
4  POINT (2681131.000 1250692.000)  
5  POIN

```python
# Der Output sollte folgendermassen aussehen:
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
```

(ex-14-3)=
### Übung 14.3

Wie ermitteln wir nun für jeden simulierten Punk, ob er im Wald ist oder nicht? In GIS Terminologie müssen wir einen sogenannten "*SpatialJoin*" durchführen, wofür es in Geopandas eine entsprechende Funktion gibt ([`sjoin`](https://geopandas.org/reference/geopandas.sjoin.html)). 

In einem *SpatialJoin* werden Attribute von dem einen Datensatz aufgrund einer räumlichen Eigenschaft auf einen anderen Datensatz übertragen. In unserem Fall: Jedem simulierten Punkt sollte die Eigenschaft "Wald: ja" / "Wald: nein" aus `wald` zugewiesen werden.

Test für dich `sjoin` (ausserhalb des Loops, siehe nachstehndes) und schaue dir den Output an, um ein Gefühl für diese Funktion zu bekommen. Integriere die Funktion anschliessend in deinen Loop indem du den Output in `sim_gpd_join` speicherst.

In [118]:
# So wird der SpatialJoin zwischen den 
# Simulierten Punkten und "Wald" durchgeführt
sim_gpd_join = gpd.sjoin(sim_gpd,wald)

In [119]:
# Musterlösung

for sim in monte_carlo:
    sim_gpd = gpd.GeoDataFrame(sim,geometry=gpd.points_from_xy(zeckenstiche['x'], zeckenstiche['y']),crs = 2056)
    sim_gpd_join = gpd.sjoin(sim_gpd,wald)

,ID,accuracy,x,y,x_sim,y_sim,geometry
0,2550,439.128951,2681116,1250648,2.681165e+06,1.250749e+06,POINT (2681116.000 1250648.000)
1,10437,301.748542,2681092,1250672,2.681109e+06,1.250676e+06,POINT (2681092.000 1250672.000)
2,9174,301.748542,2681128,1250683,2.681051e+06,1.250678e+06,POINT (2681128.000 1250683.000)
3,8773,301.748542,2681111,1250683,2.681134e+06,1.250732e+06,POINT (2681111.000 1250683.000)
4,2764,301.748529,2681131,1250692,2.680987e+06,1.250756e+06,POINT (2681131.000 1250692.000)
5,2513,301.748529,2681171,1250711,2.681024e+06,1.250663e+06,POINT (2681171.000 1250711.000)
6,9185,301.748542,2681107,1250712,2.681014e+06,1.250818e+06,POINT (2681107.000 1250712.000)
7,28521,301.748542,2681124,1250720,2.681213e+06,1.250684e+06,POINT (2681124.000 1250720.000)
8,26745,301.748542,2681117,1250725,2.681075e+06,1.250831e+06,POINT (2681117.000 1250725.000)
9,27391,301.748542,2681138,1250725,2.680907e+06,1.250966e+06,POINT (2681138.000 1250725.000)


### Übung 14.4

Nun wollen wir in jeder Iteration wissen, wie viele Zellen der Spalte "Wald_text" von `sim_gpd_join` den Wert "ja" enthalten. Dies erreichen wir, indem wir die entsprechende Spalte mit "ja" vergleichen (`sim_gpd_join["Wald_text"] == "ja"`), was für jede Zeile einen boolschen Wert `True` oder `False` zurück gibt. Die Summe von `True`-Werten erhält man über die Summe, da jedes `True` mit `1` und jedes `False` mit `0` interpretiert wird (analog R).

Integriere diese Operation in deinen Loop und speichere den Output als `anzahl_ja`.

In [94]:
sum(sim_gpd_join["Wald_text"] == "ja")

5

In [95]:
# Musterlösung

for sim in monte_carlo:
    sim_gpd = gpd.GeoDataFrame(sim,geometry=gpd.points_from_xy(zeckenstiche['x'], zeckenstiche['y']),crs = 2056)
    sim_gpd_join = gpd.sjoin(sim_gpd,wald)
    anzahl_ja = sum(sim_gpd_join["Wald_text"] == "ja")

### Übung 14.5

Wir haben nun die Anzahl der Punkte pro Interation, die im Wald sind (`anzahl_ja`). Wichtiger wäre aber der *Anteil*. Berechne der Anteil indem du `anzahl_ja` noch durch die Anzahl Zeilen (`len`) von `sim_gpd_join` dividierst. Integriere diesen Schritt in deinen Loop und speichere den Output als `anteil_ja`.

In [96]:
# Musterlösung

for sim in monte_carlo:
    sim_gpd = gpd.GeoDataFrame(sim,geometry=gpd.points_from_xy(zeckenstiche['x'], zeckenstiche['y']),crs = 2056)
    sim_gpd_join = gpd.sjoin(sim_gpd,wald)
    anzahl_ja = sum(sim_gpd_join["Wald_text"] == "ja")
    anteil_ja = anzahl_ja/len(sim_gpd_join)

### Übung 14.6

Speichere `anteil_ja` in einer Liste mit dem Namen `anteile_ja` (mehrzahl!). Schaue dir {ref}`forloops-2` an wenn du nicht mehr weisst, wie das geht.


In [105]:
ja_anteile = []
for sim in monte_carlo:
    sim_gpd = gpd.GeoDataFrame(sim,geometry=gpd.points_from_xy(zeckenstiche['x'], zeckenstiche['y']),crs = 2056)
    sim_gpd_join = gpd.sjoin(sim_gpd,wald)
    anzahl_ja = sum(sim_gpd_join["Wald_text"] == "ja")
    anteil_ja = anzahl_ja/len(sim_gpd_join)
    ja_anteile.append(anteil_ja)


    
len(monte_carlo)
monte_carlo

[      ID    accuracy        x        y         x_sim         y_sim  \
 0   2550  439.128951  2681116  1250648  2.681201e+06  1.250627e+06   
 1  10437  301.748542  2681092  1250672  2.681226e+06  1.250704e+06   
 2   9174  301.748542  2681128  1250683  2.681239e+06  1.250546e+06   
 3   8773  301.748542  2681111  1250683  2.681186e+06  1.250748e+06   
 4   2764  301.748529  2681131  1250692  2.681197e+06  1.250696e+06   
 5   2513  301.748529  2681171  1250711  2.681186e+06  1.250737e+06   
 6   9185  301.748542  2681107  1250712  2.681279e+06  1.250629e+06   
 7  28521  301.748542  2681124  1250720  2.681013e+06  1.250693e+06   
 8  26745  301.748542  2681117  1250725  2.681267e+06  1.250679e+06   
 9  27391  301.748542  2681138  1250725  2.681238e+06  1.250681e+06   
 
                           geometry  
 0  POINT (2681116.000 1250648.000)  
 1  POINT (2681092.000 1250672.000)  
 2  POINT (2681128.000 1250683.000)  
 3  POINT (2681111.000 1250683.000)  
 4  POINT (2681131.000 1250

%: Mittelwerte Visualisieren
### Übung 14.3

Gratuliere! Wenn du an diesem Punkt angekommen bist hast du eine ganze Monte Carlo Simulation von A bis Z mit Python durchgeführt. Von hier an steht dir der Weg frei für noch komplexere Analysen. Zum Abschluss kannst du die Mittelwerte wir nun auf einfache Weise visualisieren. Versuche dabei die Methods `plot()` und `plot.box()` sowie `plot.hist()`.

In [98]:
#anteil_ja.plot()

In [99]:
#anteil_ja.plot.box()

In [100]:
#anteil_ja.plot.hist()